# Init

In [2]:
from pyosys import libyosys as ys
import kagglehub
import pandas as pd
import os
import io
import tempfile
import re
import shutil

temp_dir = os.path.join(os.getcwd(), "temp_working_dir")

if os.path.exists(temp_dir):
  shutil.rmtree(temp_dir)
os.makedirs(temp_dir)

/opt/homebrew/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [3]:
path = kagglehub.dataset_download("sohamndeshmukh/verilog-code-dataset")

df = pd.read_csv(os.path.join(path, "formatted_small_df.csv"))

verilog_codes = df['Correct']
print(verilog_codes)

verilog_code = verilog_codes[0]

print(verilog_code)

0       \nmodule not_1 (\n    input a,\n    output res...
1       \nmodule nand_1 (\n    input a, b,\n    output...
2       \nmodule mux_16to1 (\n    input [15:0] d,\n   ...
3       \nmodule mux_16to1 (\n    input [15:0] d,\n   ...
4       /*\n * Copyright 2012, Homer Hsing <homer.hsin...
                              ...                        
1935    \nmodule mux_8to1 (\n    input [7:0] d,\n    i...
1936    \nmodule or_1 (\n    input a, b,\n    output r...
1937    \nmodule nor_1 (\n    input a, b,\n    output ...
1938    \nmodule or_1 (\n    input a, b,\n    output r...
1939    \nmodule mux_8to1 (\n    input [7:0] d,\n    i...
Name: Correct, Length: 1940, dtype: object

module not_1 (
    input a,
    output result
);
    assign result = ~a;
endmodule



# Generate blif using yosys

In [4]:
match = re.search(r'\bmodule\s+(\w+)', verilog_code)
if match:
  module_name = match.group(1)
else:
  raise Exception("Cannot get module name!")

original_verilog_code_path = os.path.join(temp_dir, "v_original.v")
with open(original_verilog_code_path, "w") as f:
  f.write(verilog_code)

design = ys.Design()
ys.run_pass("read_verilog " + original_verilog_code_path, design)
ys.run_pass("synth -top " + module_name, design)

out_file = os.path.join(temp_dir, "out_blif.blif")

ys.run_pass("write_blif " + out_file, design)

with open(out_file, "r") as file:
  blif_content = file.read()


-- Running command `read_verilog /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/v_original.v' --

1. Executing Verilog-2005 frontend: /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/v_original.v
Parsing Verilog input from `/Users/gabi/Documents/hwdecomp-llm/temp_working_dir/v_original.v' to AST representation.
Generating RTLIL representation for module `\not_1'.
Successfully finished Verilog frontend.

-- Running command `synth -top not_1' --

2. Executing SYNTH pass.

2.1. Executing HIERARCHY pass (managing design hierarchy).

2.1.1. Analyzing design hierarchy..
Top module:  \not_1

2.1.2. Analyzing design hierarchy..
Top module:  \not_1
Removed 0 unused modules.

2.2. Executing PROC pass (convert processes to netlists).

2.2.1. Executing PROC_CLEAN pass (remove empty switches from decision trees).
Cleaned up 0 empty switches.

2.2.2. Executing PROC_RMDEAD pass (remove dead branches from decision trees).
Removed a total of 0 dead cases.

2.2.3. Executing PROC_PRUNE pass (rem

# Recover Verilog code using codellama

In [5]:
import ollama

response: ollama.ChatResponse = ollama.chat(model='codellama:7b', messages=[
 {
   'role': 'user',
   'content': 'Convert the following BLIF code to Verilog (not SystemVerilog). Use only standard Verilog-2001 syntax and keep the code as simple and minimal as possible.\n```blif\n' + blif_content + '\n```'
 }
])

print(response.message.content)

The given BLIF code is a model for a NOT gate. Here's the equivalent Verilog code:
```verilog
module not_1(a, result);
    input a;
    output result;
    
    always @(*) begin
        result = !a;
    end
endmodule
```
Explanation:

* The `not_1` module is declared with an input port `a` and an output port `result`.
* The `always` block is used to implement the NOT logic. The `!` operator is used to flip the value of `a`.
* The `begin` and `end` keywords are used to define a procedural block, which is necessary for the `always` keyword.
* The `@(*)` syntax is used to specify that the block should be triggered by any change in the inputs.


In [6]:
pattern = r"```verilog\s+(.*?)```"
match = re.search(pattern, response.message.content, re.DOTALL)
if match:
  verilog_code_recovered = match.group(1).strip()
  print(verilog_code_recovered)
else:
  print("No matches")

module not_1(a, result);
    input a;
    output result;
    
    always @(*) begin
        result = !a;
    end
endmodule


# Verification

In [7]:
import subprocess
import shutil

EQY_PATH = '/Users/gabi/oss-cad-suite/bin/eqy'

eqy_testing = False

if eqy_testing:
  eqy_working_dir = os.path.join(temp_dir, "eqy_working_dir")
  if os.path.exists(eqy_working_dir):
    shutil.rmtree(eqy_working_dir)

  result = subprocess.run([EQY_PATH, 'eq_test.eqy', '-d', eqy_working_dir], capture_output=True, text=True)
  print("Return code:", result.returncode)
  print("Output:\n", result.stdout)
  print("Error output:\n", result.stderr)

  if result.returncode == 0:
    print("verification PASSED")
  else:
    print("verification FAILED")



In [8]:
import subprocess
import shutil

recovered_verilog_code_path = os.path.join(temp_dir, "v_recovered.v")
with open(recovered_verilog_code_path, "w") as f:
  f.write(verilog_code_recovered)

# Create an eqy file for equivalence checking
eqy_content = f"""
[gold]
read_verilog {original_verilog_code_path}
prep -top {module_name}

[gate]
read_verilog {recovered_verilog_code_path}
prep -top {module_name}

[strategy simple]
use sby
"""

eqy_config_path = os.path.join(temp_dir, "eqy_config.eqy")
with open(eqy_config_path, "w") as f:
  f.write(eqy_content)

# Run EQY verification

eqy_working_dir = os.path.join(temp_dir, "eqy_working_dir")
if os.path.exists(eqy_working_dir):
  shutil.rmtree(eqy_working_dir)

result_recovered = subprocess.run([EQY_PATH, eqy_config_path, '-d', eqy_working_dir], capture_output=True, text=True)
print("Return code:", result_recovered.returncode)
print("Output:\n", result_recovered.stdout)
print("Error output:\n", result_recovered.stderr)

if result_recovered.returncode == 0:
  print("verification PASSED")
else:
  print("verification FAILED")

Return code: 0
Output:
 EQY  6:35:38 [/Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir] read_gold: starting process "yosys -ql /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir/gold.log /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir/gold.ys"
EQY  6:35:39 [/Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir] read_gold: finished (returncode=0)
EQY  6:35:39 [/Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir] read_gate: starting process "yosys -ql /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir/gate.log /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir/gate.ys"
EQY  6:35:39 [/Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir] read_gate: Warning: wire '\result' is assigned in a block at /Users/gabi/Documents/hwdecomp-llm/temp_working_dir/v_recovered.v:6.9-6.20.
EQY  6:35:39 [/Users/gabi/Documents/hwdecomp-llm/temp_working_dir/eqy_working_dir] rea

In [9]:
shutil.rmtree(temp_dir)